In [ ]:
import folium
import pandas as pd

processed_cdr_df = pd.read_pickle("datasets/processed_cdr.pkl")
# Sample the processed data
sample_df = processed_cdr_df.sample(500)  # adjust N

# Create a folium map centered on the data
m = folium.Map(location=[38.7169, -9.1399], zoom_start=13)

# Add estimated points
for _, row in sample_df.iterrows():
    folium.CircleMarker(
        location=[row["est_lat"], row["est_lon"]],
        radius=2,
        color="blue",
        fill=True,
        fill_opacity=0.5,
        popup=f"user: {row['unique_id']}, cell: {row['cell_id']}"
    ).add_to(m)

# Optionally add tower locations
towers = sample_df[["longitude_cell", "latitude_cell", "cell_id"]].drop_duplicates()
for _, row in towers.iterrows():
    folium.Marker(
        location=[row["latitude_cell"], row["longitude_cell"]],
        icon=folium.Icon(color="red", icon="signal"),
        popup=f"Tower: {row['cell_id']}"
    ).add_to(m)

m.save("tower_spread_map_3.html")
